In [1]:
import os

from QHyper.problems.algorithms.solver_utils import WorkflowSchedulingSolverDecorator, \
    DecomposedWorkflowSchedulingSolver
from QHyper.problems.algorithms.workflow_decomposition import HeftBasedAlgorithm, add_entry_and_exit_tasks, \
    SeriesParallelSplit
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingOneHot
from QHyper.solvers import solver_from_config
from QHyper.solvers.gurobi import Gurobi
from reports.report import ExecutionReport, Solution

In [2]:
def get_solver(tasks_file: str, machines_file: str, deadline: int):
    solver_config = {
        "problem": {
            "type": "workflow_scheduling",
            "encoding": "one-hot",
            "tasks_file": tasks_file,
            "machines_file": machines_file,
            "deadline": deadline,
        },
        "solver": {
            "type": "gurobi",
        }
    }

    return WorkflowSchedulingSolverDecorator(solver_from_config(solver_config))


def get_filename(tasks_file: str):
    workflow_name, _ = (tasks_file.split(sep="/")[-1]).split(sep=".")
    return f"gurobi_{workflow_name}.json"


def run_tests(test_data, save_dir):
    for tasks_file, machines_file, deadline in test_data:
        print(f"Executing test for {tasks_file}...")
        gurobi = get_solver(tasks_file, machines_file, deadline)
        schedule = gurobi.solve()
        report = ExecutionReport(
            workflow_file=tasks_file,
            machines_file=machines_file,
            deadline=deadline,
            solver="Gurobi",
            solution=Solution.from_workflow_schedule(schedule)
        )
        report.write_json(os.path.join(save_dir, get_filename(tasks_file)))

In [3]:
tasks_files = [
    "workflows_data/workflows/srasearch_22_tasks.json",
    # "workflows_data/workflows/1000genome_156_tasks.json",
    # "workflows_data/workflows/1000genome_492_tasks.json",
    # "workflows_data/workflows/1000genome_902_tasks.json"
]

machines_files = [
    # "workflows_data/machines/linear.json",
    # "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json",
    # "workflows_data/machines/linear.json"
]

deadlines = [
    # 5000,
    # 5000,
    3000,
    # 5000
]

test_data = zip(tasks_files, machines_files, deadlines)

In [4]:
run_tests(test_data, "reports/linear_machines/3000/hurbol")

Executing test for workflows_data/workflows/srasearch_22_tasks.json...
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-12
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)
CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42 rows, 110 columns and 410 nonzeros
Model fingerprint: 0xfbb8a767
Variable types: 0 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+04]
  Objective range  [1e+00, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective 328338.81051
Presolve removed 42 rows and 110 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 286273 328339 

Opti

In [16]:
def get_decomposed_solver(tasks_file: str, machines_file: str, deadline: int, max_graph_size: int):
    workflow = Workflow(tasks_file, machines_file, deadline)
    division = SeriesParallelSplit().decompose(workflow, max_graph_size)
    problems = map(lambda w: WorkflowSchedulingOneHot(w), division.workflows)
    solvers = map(lambda p: WorkflowSchedulingSolverDecorator(Gurobi(p)), problems)
    return DecomposedWorkflowSchedulingSolver(list(solvers), division)


def get_filename(tasks_file: str):
    workflow_name, _ = (tasks_file.split(sep="/")[-1]).split(sep=".")
    return f"gurobi_{workflow_name}.json"


def run_tests_division(test_data, save_dir, max_graph_size):
    for tasks_file, machines_file, deadline in test_data:
        print(f"Executing test for {tasks_file}...")
        solver = get_decomposed_solver(tasks_file, machines_file, deadline, max_graph_size)
        schedule = solver.solve()
        report = ExecutionReport(
            workflow_file=tasks_file,
            machines_file=machines_file,
            deadline=deadline,
            solver="Gurobi",
            solution=schedule,
            max_graph_size=max_graph_size
        )
        report.write_json(os.path.join(save_dir, get_filename(tasks_file)))

In [12]:
tasks_files = [
    "workflows_data/workflows/srasearch_22_tasks.json",
    "workflows_data/workflows/1000genome_156_tasks.json",
    "workflows_data/workflows/1000genome_492_tasks.json",
    "workflows_data/workflows/1000genome_902_tasks.json"
]

machines_files = [
    "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json"
]

deadlines = [
    3000,
    3000,
    3000,
    3000
]

test_data = zip(tasks_files, machines_files, deadlines)

In [13]:
run_tests_division(test_data, "reports/linear_machines/3000/my", 15)

Executing test for workflows_data/workflows/srasearch_22_tasks.json...
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 24 rows, 65 columns and 120 nonzeros
Model fingerprint: 0xdc0836ac
Variable types: 0 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [8e+01, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+03]
Found heuristic solution: objective 233566.94525
Presolve removed 24 rows and 65 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 111416 233567 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.114159353216e+05, best bound 1.1141593

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

